In [1]:
import numpy as np
from coverage import gen_sats, gen_times, camera_model, get_inst_fov, get_inst_fov2, los_to_earth, get_lvlh_pointing
from datetime import datetime, timezone, timedelta
import dataclasses
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import branca
import folium

from skyfield.api import load, wgs84, Distance
from skyfield.framelib import itrs
from skyfield.toposlib import ITRSPosition
from vanzyl1 import Instrument, Platform, Scene

from shapely.geometry import Polygon, Point

from scipy.spatial.transform import Rotation

In [2]:
start_dt = datetime.fromisoformat(Scene.start_utc)
num_days = 8

tles = gen_sats(
    sat_nos=[Platform.norad_id] # How to best handle multiple platforms? (TLE vs. SPG4 model too)
    # sat_nos=[39084]
    # sat_nos=[39084,49260]
)

inst = camera_model(
    name=Instrument.name, 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=start_dt.year,
    start_mo=start_dt.month, 
    start_day=start_dt.day, 
    days=num_days, 
    step_min=Instrument.img_period)

xcell_size = ycell_size = .1

Satellite(s) Loaded from TLE:

[<EarthSatellite YAM-3 catalog #48915 epoch 2022-07-27 10:40:10 UTC>]

Propogation time: 
 2022-07-30 00:00:00+00:00 
to 
 2022-08-06 23:59:58.500000+00:00

In [3]:
sat = tles[0][0]

In [4]:
# %%timeit
# get_inst_fov(sat, times[0], inst)

In [5]:
llas = get_inst_fov(sat, times[0], inst)

In [6]:
# %%timeit
# get_inst_fov2(sat, times[0], inst)

In [7]:
llas2 = get_inst_fov2(sat, times[0], inst)

In [8]:
llas2

{'c1': {'lat': -60.72997494342293, 'lon': -160.37714856885114},
 'c2': {'lat': -60.529695645646854, 'lon': -161.6919784978892},
 'c3': {'lat': -61.17442172842571, 'lon': -162.12146365150215},
 'c4': {'lat': -61.378942425703485, 'lon': -160.78189209775826}}

In [9]:
llas

{'c1': {'lat': -60.72997494342293, 'lon': -160.37714856885114},
 'c2': {'lat': -60.529695645646854, 'lon': -161.6919784978892},
 'c3': {'lat': -61.17442172842571, 'lon': -162.12146365150215},
 'c4': {'lat': -61.378942425703485, 'lon': -160.78189209775826}}

In [10]:
c1 = np.array(
    [-0.017*(1024/2), 0.017*(1024/2), 126]
    )
c2 = np.array(
    [0.017*(1024/2), 0.017*(1024/2), 126]
    )
c3 = np.array(
    [0.017*(1024/2), -0.017*(1024/2), 126]
    )
c4 = np.array(
    [-0.017*(1024/2), -0.017*(1024/2), 126]
    )
c1 = c1/ np.linalg.norm(c1)
c2 = c2/ np.linalg.norm(c2)
c3 = c3/ np.linalg.norm(c3)
c4 = c4/ np.linalg.norm(c4)

In [14]:
# %%timeit

lvlh = get_lvlh_pointing(sat, times[0])
lvlh = np.array([lvlh["X"], lvlh["Y"], lvlh["Z"]])

camera_frame = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1]
    ])

rot, rms = Rotation.align_vectors(lvlh, camera_frame)
xyz_dist_rates = sat.at(times[0]).frame_xyz_and_velocity(itrs)
xyz_dist = xyz_dist_rates[0]
z_rate = xyz_dist_rates[1]

points = []

los_XY = rot.apply([c1, c2, c3, c4])

# 946 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each) - up to here

for los in los_XY:
    
    los_xyz = los_to_earth(xyz_dist.km, los)
    los_itrs = ITRSPosition(Distance(km=los_xyz))
    # Convert intercept lat/ lon from ITRS frame
    los_lat, los_lon = wgs84.latlon_of(los_itrs.at(times[0]))

    points.append([los_lon.degrees, los_lat.degrees])
    # print(los_lat.degrees, los_lon.degrees)

    # 2.99 ms ± 360 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) - includes above

poly1 = Polygon(points)

# 2.93 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) - total for cell

In [15]:
poly2 = Polygon(
[
    (llas["c1"]["lon"], llas["c1"]["lat"]),
    (llas["c2"]["lon"], llas["c2"]["lat"]),
    (llas["c3"]["lon"], llas["c3"]["lat"]),
    (llas["c4"]["lon"], llas["c4"]["lat"]),
    (llas["c1"]["lon"], llas["c1"]["lat"]),
]
)

In [16]:
gpd.GeoDataFrame(data=[poly1, poly2], columns=["geometry"], crs="EPSG:4326").explore()